In [1]:
import os

In [2]:
%pwd

'/home/priyanshu1303d/Projects/Chicken_Disease_Classification/research'

In [3]:
# as we want to go to the working directory instead of research dir

os.chdir("../")

In [4]:
%pwd

'/home/priyanshu1303d/Projects/Chicken_Disease_Classification'

In [7]:
# Update the entity so for that we are creating our own class
#entity is just the return type of a class.. so here we are creating our return type manually
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
#Frozen helps us such that is the object i initialized have source_url as "https://abc.com" and later on i try to change the attribute of the source_url to "https://cba.com" it will throw error as FrozenInstanceError

class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        # here we are getting config as ConfigBox type



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [10]:
#Update the Components
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded!!! wiht following into : \n{headers}")
        else:
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [21]:
# Create the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-01-03 12:38:15,381: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-01-03 12:38:15,384: INFO: common : yaml file: params.yaml loaded successfully]
[2025-01-03 12:38:15,385: INFO: common : created directory at: artifacts]
[2025-01-03 12:38:15,386: INFO: common : created directory at: artifacts/data_ingestion]
[2025-01-03 12:38:26,374: INFO: 905255374 : artifacts/data_ingestion/data.zip downloaded!!! wiht following into : 
Connection: close
Content-Length: 18192175
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "422adfad34d555b49724f1beb7681db39d80733f91e4419a69e7250180f31a77"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9392:1D2AD0:9113CF:D63D9A:67778B01
Accept-Ranges: bytes
Date: Fri, 03 Jan 2025 07:08:21 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100020-HY